In [1]:
#URI: gs://pdf168/Resume.pdf

In [17]:
import os, io
import re
from google.cloud import vision
from google.cloud import storage #!pip install --upgrade google-cloud-storage
from google.protobuf import json_format

In [18]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="../../key.json"
client=vision.ImageAnnotatorClient()

### 1) Define the feature

In [19]:
batch_size=1 #How many page
mime_type= 'application/pdf'#file type

#Initiate a feature object
feature = vision.types.Feature(
    type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION
)

### 2) Define the input, output and create a json file for it

Define

In [20]:
#Input
gcs_source_uri="gs://pdf168/Resume.pdf" #Uniform resource identifier
gcs_source=vision.types.GcsSource(uri=gcs_source_uri) 

#Mime_type: The type of the file. Currently only 
#"application/pdf", "image/tiff" and "image/gif" are supported.

input_config=vision.types.InputConfig(
    gcs_source=gcs_source, 
    mime_type=mime_type
)

In [38]:
#Output
file_name="pdf_result"
gcs_destination_uri="gs://pdf168/pdf_result"  
gcs_destination=vision.types.GcsDestination(uri=gcs_destination_uri)

output_config=vision.types.OutputConfig(
    gcs_destination=gcs_destination,
    batch_size=1 #Have to do with the number of json file created
)

Create

In [39]:
#An offline file annotation request.
#AsyncAnnotate: asynchronous (at the same time) image detection and annotation for a list of generic files
async_request=vision.types.AsyncAnnotateFileRequest(
    features=[feature], #Here is type: DOCUMENT_TEXT_DETECTION
    input_config=input_config,
    output_config=output_config
)

operation=client.async_batch_annotate_files(requests=[async_request])

### 3) Retrieve the offline version of the json file

In [40]:
# Once the request has completed and the output has been
# written to GCS, we can list all the output files.
storage_client=storage.Client()

#Check if the uri is in a certain form
match=re.match('gs://([^/]+)/(.+)', gcs_destination_uri) 
bucket_name=match.group(1) #extract the first bracket (gcs folder name)
prefix=match.group(2) #extract the second bracket (output file name)
bucket=storage_client.get_bucket(bucket_name)

In [41]:
#List object with the given prefix 
#Note that the prefix here is the name of our output
blob_list=list(bucket.list_blobs(prefix=prefix))
print('Output files')
for blob in blob_list:
    print(blob.name)

Output files
pdf_resultoutput-1-to-1.json


In [44]:
blob_list

[<Blob: pdf168, pdf_resultoutput-1-to-1.json, 1580543442820372>]

In [45]:
# Process the first output file from GCS. There is only 1 output in 
# this case

output=blob_list[0]
json_string = output.download_as_string()
response=json_format.Parse(
    json_string, vision.types.AnnotateFileResponse()
)

In [46]:
# The actual response for the first page of the input file
first_page_response=response.responses[0]
annotation=first_page_response.full_text_annotation

In [47]:
# Here we print the full text from the first page.
# The response contains more information:
# annotation/pages/blocks/paragraphs/words/symbols
# including confidence scores and bounding boxes
print(annotation.text)

HUY BUI
Phone: 737-202-9999| Email: williamhuybui@gmail.com​| GitHub: williamhuybui | Website: github.io​ | LinkedIn: Huy Bui
Data scientist and mathematician who enjoys wrangling with data and discovering new insights. Experience in producing statistical analysis, data visualization, advanced predictive models, and neural networks. With a background in teaching and studying advanced mathematics, I can communicate analytical results to both technical and non-technical audiences. I am a quick learner who has strong attention to detail and work well in a team.
Python, data wrangling (Pandas, Numpy), data exploration (matplotlib, Tableau, PowerBI), machine learning (Scikit-learn), neural networks (PyTorch, TensorFlow, Keras), SQL, NLTK, web scraping (BeautifulSoup, Selenium), LaTex, C++, MatLab Fluent in Vietnamese (native)
Predicting Early Childhood Learning Outcome through PBS Kids Measure Up + Reduced 11 million observations to 14 thousand by strategically grouping, aggregating, and cl